## Inserting data in MySQL using Python

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
import requests

In [2]:
# Let's get the most recent data from the Citibike API
url = 'http://www.citibikenyc.com/stations/json'
results = requests.get(url).json() 
print(results)

{'executionTime': '2019-09-01 02:57:15 PM', 'stationBeanList': [{'id': 298, 'stationName': '3 Ave & Schermerhorn St', 'availableDocks': 9, 'totalDocks': 35, 'latitude': 40.68683208, 'longitude': -73.9796772, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 24, 'stAddress1': '3 Ave & Schermerhorn St', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2019-09-01 02:56:45 PM', 'landMark': ''}, {'id': 301, 'stationName': 'E 2 St & Avenue B', 'availableDocks': 39, 'totalDocks': 58, 'latitude': 40.72217444, 'longitude': -73.98368779, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 17, 'stAddress1': 'E 2 St & Avenue B', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2019-09-01 02:55:58 PM', 'landMark': ''}, {'id': 337, 'stationName': 'Old Slip & Front St', 'availableDocks': 16, 'totalDocks': 37, 'latitude': 

In [3]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only we need
print(results.keys())
print()
data = results["stationBeanList"]
print(data[0:3])

dict_keys(['executionTime', 'stationBeanList'])

[{'id': 298, 'stationName': '3 Ave & Schermerhorn St', 'availableDocks': 9, 'totalDocks': 35, 'latitude': 40.68683208, 'longitude': -73.9796772, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 24, 'stAddress1': '3 Ave & Schermerhorn St', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2019-09-01 02:56:45 PM', 'landMark': ''}, {'id': 301, 'stationName': 'E 2 St & Avenue B', 'availableDocks': 39, 'totalDocks': 58, 'latitude': 40.72217444, 'longitude': -73.98368779, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 17, 'stAddress1': 'E 2 St & Avenue B', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2019-09-01 02:55:58 PM', 'landMark': ''}, {'id': 337, 'stationName': 'Old Slip & Front St', 'availableDocks': 16, 'totalDocks': 37, 'latitude': 40.7037992, 'l

In [4]:
len(data)

843

Now we will connect to our MySQL server. We will use the MySQLdb library of Python.

If you do not have the library, you need to install it by typing in the shell:

`sudo apt-get install python-mysqldb`
# NOTE: This code doesn't work on your docker image
# We need to connect bigdata.stern.nyu.edu

In [12]:
import MySQLdb as mdb

#con = mdb.connect(host = 'localhost', 
#                  user = 'root', 
#                  passwd = 'dwdstudent2015', 
#                  charset='utf8', use_unicode=True);


con = mdb.connect(host = 'bigdata.stern.nyu.edu', 
                  user = 'DealingF19',                  
                  passwd = 'DealingF19!!', 
                  charset='utf8', use_unicode=True);

Once we have connected successfully, we need to create our database:

In [ ]:
# Query to create a database
db_name = 'citibike_mysql_test'
# Need to drop database since the not exists option doesn't sem to work
db_drop ="drop database {db} ".format(db=db_name)
cursor=con.cursor()
cursor.execute(db_drop)
cursor.close()

In [14]:
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

Then we create the table where we will store our data. For our example, we will just import three fields in the database: station_id, station_name, and number_of_docks

In [15]:
cursor = con.cursor()
table_name = 'Docks'
# Create a table
# Drop it if it exists
#create_drop_docks=''' drop table {db}.{table}'''.format(db=db_name,table=table_name)
#cursor.execute(create_drop_docks)

# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (station_id int, 
                                station_name varchar(250), 
                                number_of_docks int,
                                available_docks int,
                                date datetime,
                                PRIMARY KEY(station_id, date)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

Finally, we import the data into our table, using the INSERT command. 

In [16]:
from datetime import date, datetime, timedelta


query_template = '''INSERT INTO {db}.{table}(station_id, 
                                            station_name, 
                                            number_of_docks, 
                                            available_docks, 
                                            date) 
                    VALUES (%s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

# THIS IS PROHIBITED
# query = "INSERT INTO citibike.Docks(station_id, station_name, number_of_docks) VALUES ("+entry["id"]+", "+entry["stationName"]+", "+entry["totalDocks"]+")"
for entry in data:
    dockid = entry["id"]
    addr = entry["stationName"]
    docks = entry["totalDocks"]
    available = entry["availableDocks"]
    # date =  datetime.now()
    # lastcommunicationtime is a string of 
    # the form "2016-02-09 10:16:49 AM"
    # See https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
    # to see the documentation on how to parse 
    date = datetime.strptime(entry["lastCommunicationTime"], 
                             '%Y-%m-%d %I:%M:%S %p')
    print("Inserting station", dockid, "at", addr)
    query_parameters = (dockid, addr, docks, available, date)
    cursor.execute(query_template, query_parameters)
# commit the changes

con.commit()
cursor.close()

Inserting station 298 at 3 Ave & Schermerhorn St
Inserting station 301 at E 2 St & Avenue B
Inserting station 337 at Old Slip & Front St
Inserting station 358 at Christopher St & Greenwich St
Inserting station 426 at West St & Chambers St
Inserting station 432 at E 7 St & Avenue A
Inserting station 438 at St Marks Pl & 1 Ave
Inserting station 514 at 12 Ave & W 40 St
Inserting station 532 at S 5 Pl & S 5 St
Inserting station 3255 at 8 Ave & W 31 St
Inserting station 3467 at W Broadway & Spring Street
Inserting station 72 at W 52 St & 11 Ave
Inserting station 79 at Franklin St & W Broadway
Inserting station 82 at St James Pl & Pearl St
Inserting station 83 at Atlantic Ave & Fort Greene Pl
Inserting station 116 at W 17 St & 8 Ave
Inserting station 119 at Park Ave & St Edwards St
Inserting station 120 at Lexington Ave & Classon Ave
Inserting station 127 at Barrow St & Hudson St
Inserting station 128 at MacDougal St & Prince St
Inserting station 143 at Clinton St & Joralemon St
Inserting st

Inserting station 3159 at W 67 St & Broadway
Inserting station 3160 at Central Park West & W 76 St
Inserting station 3161 at W 76 St & Columbus Ave
Inserting station 3162 at W 78 St & Broadway
Inserting station 3163 at Central Park West & W 68 St
Inserting station 3164 at Columbus Ave & W 72 St
Inserting station 3165 at Central Park West & W 72 St
Inserting station 3166 at Riverside Dr & W 72 St
Inserting station 3167 at Amsterdam Ave & W 73 St
Inserting station 3168 at Central Park West & W 85 St
Inserting station 3169 at Riverside Dr & W 82 St
Inserting station 3170 at W 84 St & Columbus Ave
Inserting station 3171 at Amsterdam Ave & W 82 St
Inserting station 3172 at W 74 St & Columbus Ave
Inserting station 3173 at Riverside Blvd & W 67 St
Inserting station 3175 at W 70 St & Amsterdam Ave
Inserting station 3177 at W 84 St & Broadway
Inserting station 3178 at Riverside Dr & W 78 St
Inserting station 3179 at Park Ave & Marcus Garvey Blvd
Inserting station 3182 at Yankee Ferry Terminal
I

Inserting station 3616 at Steinway St & 28 Ave
Inserting station 3617 at 28 Ave & 35 St
Inserting station 3618 at 27 St & Hunter St
Inserting station 3619 at Newtown Ave & 23 St
Inserting station 3620 at 11 St & 43 Ave
Inserting station 3621 at 27 Ave & 9 St
Inserting station 3622 at E 128 St & Madison Ave
Inserting station 3623 at W 120 St & Claremont Ave
Inserting station 3628 at Lenox Ave & W 117 St
Inserting station 3629 at Adam Clayton Powell Blvd & W 126 St
Inserting station 3630 at Frederick Douglass Blvd & W 115 St
Inserting station 3637 at Fulton St & Waverly Ave
Inserting station 3638 at Washington St
Inserting station 3639 at Harborside
Inserting station 3640 at Journal Square
Inserting station 3641 at Broadway & W 25 St
Inserting station 3646 at 35 Ave & 10 St
Inserting station 3647 at 48 Ave & 30 Pl
Inserting station 3648 at Flushing Ave & Vanderbilt Ave
Inserting station 3649 at W 129 St & Convent Ave
Inserting station 3654 at 31 St & Northern Blvd
Inserting station 3656 

In [17]:
cur = con.cursor(mdb.cursors.DictCursor)
cur.execute("SELECT * FROM {db}.{table}".format(db=db_name, table=table_name))
print(cur.execute)
rows = cur.fetchall()
cur.close()

<bound method BaseCursor.execute of <MySQLdb.cursors.DictCursor object at 0x2b5ac10c28d0>>


In [18]:
for row in rows:
    print("Station ID:", row["station_id"])
    print("Station Name:", row["station_name"])
    print("Number of Docks:", row["number_of_docks"])
    print("Available Docks:", row["available_docks"])
    print("Last Communication:", row["date"])
    print("=============================================")
    


Station ID: 72
Station Name: W 52 St & 11 Ave
Number of Docks: 55
Available Docks: 50
Last Communication: 2019-09-01 14:56:37
Station ID: 79
Station Name: Franklin St & W Broadway
Number of Docks: 33
Available Docks: 24
Last Communication: 2019-09-01 14:56:49
Station ID: 82
Station Name: St James Pl & Pearl St
Number of Docks: 27
Available Docks: 0
Last Communication: 2019-09-01 14:57:09
Station ID: 83
Station Name: Atlantic Ave & Fort Greene Pl
Number of Docks: 62
Available Docks: 11
Last Communication: 2019-09-01 14:57:13
Station ID: 116
Station Name: W 17 St & 8 Ave
Number of Docks: 39
Available Docks: 36
Last Communication: 2019-09-01 14:56:32
Station ID: 119
Station Name: Park Ave & St Edwards St
Number of Docks: 19
Available Docks: 10
Last Communication: 2019-09-01 14:53:36
Station ID: 120
Station Name: Lexington Ave & Classon Ave
Number of Docks: 19
Available Docks: 18
Last Communication: 2019-09-01 14:56:54
Station ID: 127
Station Name: Barrow St & Hudson St
Number of Docks: 31

Last Communication: 2019-09-01 14:56:41
Station ID: 473
Station Name: Rivington St & Chrystie St
Number of Docks: 38
Available Docks: 16
Last Communication: 2019-09-01 14:56:48
Station ID: 474
Station Name: 5 Ave & E 29 St
Number of Docks: 56
Available Docks: 55
Last Communication: 2019-09-01 14:56:50
Station ID: 476
Station Name: E 31 St & 3 Ave
Number of Docks: 47
Available Docks: 41
Last Communication: 2019-09-01 14:56:53
Station ID: 477
Station Name: W 41 St & 8 Ave
Number of Docks: 59
Available Docks: 53
Last Communication: 2019-09-01 14:56:49
Station ID: 478
Station Name: 11 Ave & W 41 St
Number of Docks: 37
Available Docks: 35
Last Communication: 2019-09-01 14:54:58
Station ID: 479
Station Name: 9 Ave & W 45 St
Number of Docks: 31
Available Docks: 25
Last Communication: 2019-09-01 14:55:49
Station ID: 480
Station Name: W 53 St & 10 Ave
Number of Docks: 29
Available Docks: 28
Last Communication: 2019-09-01 14:56:41
Station ID: 481
Station Name: S 3 St & Bedford Ave
Number of Dock

Number of Docks: 22
Available Docks: 14
Last Communication: 2019-09-01 14:55:56
Station ID: 3213
Station Name: Van Vorst Park
Number of Docks: 21
Available Docks: 18
Last Communication: 2019-09-01 14:53:57
Station ID: 3214
Station Name: Essex Light Rail
Number of Docks: 22
Available Docks: 4
Last Communication: 2019-09-01 14:56:19
Station ID: 3220
Station Name: 5 Corners Library
Number of Docks: 18
Available Docks: 8
Last Communication: 2019-09-01 14:56:08
Station ID: 3221
Station Name: 47 Ave & 31 St
Number of Docks: 27
Available Docks: 10
Last Communication: 2019-09-01 14:54:23
Station ID: 3225
Station Name: Baldwin at Montgomery
Number of Docks: 14
Available Docks: 10
Last Communication: 2019-09-01 14:56:24
Station ID: 3231
Station Name: E 67 St & Park Ave
Number of Docks: 39
Available Docks: 28
Last Communication: 2019-09-01 14:56:46
Station ID: 3232
Station Name: Bond St & Fulton St
Number of Docks: 31
Available Docks: 11
Last Communication: 2019-09-01 14:56:21
Station ID: 3233
St

Number of Docks: 29
Available Docks: 12
Last Communication: 2019-09-01 14:55:48
Station ID: 3507
Station Name: Park Ave & E 124 St
Number of Docks: 36
Available Docks: 17
Last Communication: 2019-09-01 14:53:30
Station ID: 3508
Station Name: St Nicholas Ave & Manhattan Ave
Number of Docks: 31
Available Docks: 24
Last Communication: 2019-09-01 14:56:03
Station ID: 3509
Station Name: Lenox Ave & W 115 St
Number of Docks: 31
Available Docks: 4
Last Communication: 2019-09-01 14:56:44
Station ID: 3510
Station Name: Adam Clayton Powell Blvd & W 123 St
Number of Docks: 31
Available Docks: 7
Last Communication: 2019-09-01 14:56:57
Station ID: 3511
Station Name: Adam Clayton Powell Blvd & W 115 St
Number of Docks: 22
Available Docks: 7
Last Communication: 2019-09-01 14:55:39
Station ID: 3512
Station Name: 27 Ave & 4 St
Number of Docks: 0
Available Docks: 0
Last Communication: 2019-08-29 13:43:50
Station ID: 3513
Station Name: 21 St & Hoyt Ave S
Number of Docks: 21
Available Docks: 5
Last Commun

Number of Docks: 22
Available Docks: 22
Last Communication: 2019-09-01 14:54:29
Station ID: 3788
Station Name: E 12 St & 4 Av
Number of Docks: 27
Available Docks: 23
Last Communication: 2019-09-01 14:55:44
Station ID: 3789
Station Name: Fulton St & Irving Pl
Number of Docks: 31
Available Docks: 29
Last Communication: 2019-09-01 14:56:29
Station ID: 3790
Station Name: Henry St & Middagh St
Number of Docks: 37
Available Docks: 10
Last Communication: 2019-09-01 14:54:57
Station ID: 3791
Station Name: Hoboken Ave at Monmouth St
Number of Docks: 32
Available Docks: 17
Last Communication: 2019-09-01 14:56:43
Station ID: 3792
Station Name: Columbus Dr at Exchange Pl
Number of Docks: 29
Available Docks: 11
Last Communication: 2019-09-01 14:53:48
Station ID: 3795
Station Name: 10 St & 2 Ave
Number of Docks: 19
Available Docks: 1
Last Communication: 2019-09-01 14:55:58
Station ID: 3798
Station Name: W 40 St & 5 Ave
Number of Docks: 55
Available Docks: 40
Last Communication: 2019-09-01 14:56:09
S

We can, of course, transform the results back into a DataFrame (see below) or we can use the data directly from the rows object (which is a tuple, containing one dictionary object for each line of the results).

In [19]:
import pandas as pd
cur = con.cursor(mdb.cursors.DictCursor)
cur.execute("SELECT * FROM {db}.{table}".format(db=db_name, table=table_name))
rows = cur.fetchall()
cur.close()

In [20]:
df_from_sql = pd.DataFrame(list(rows))
df_from_sql

,station_id,station_name,number_of_docks,available_docks,date
0,72,W 52 St & 11 Ave,55,50,2019-09-01 14:56:37
1,79,Franklin St & W Broadway,33,24,2019-09-01 14:56:49
2,82,St James Pl & Pearl St,27,0,2019-09-01 14:57:09
3,83,Atlantic Ave & Fort Greene Pl,62,11,2019-09-01 14:57:13
4,116,W 17 St & 8 Ave,39,36,2019-09-01 14:56:32
5,119,Park Ave & St Edwards St,19,10,2019-09-01 14:53:36
6,120,Lexington Ave & Classon Ave,19,18,2019-09-01 14:56:54
7,127,Barrow St & Hudson St,31,29,2019-09-01 14:53:30
8,128,MacDougal St & Prince St,30,20,2019-09-01 14:56:57
9,143,Clinton St & Joralemon St,24,18,2019-09-01 14:56:01


In [21]:
# We can then compute functions directly on the dataframe
sum(df_from_sql["number_of_docks"])

26451

In [22]:
# We can then compute functions directly on the dataframe
sum(df_from_sql["available_docks"])

15803

In [23]:
# And we can also create 
df_from_sql["bikes_docked"] = df_from_sql["number_of_docks"] - df_from_sql["available_docks"]

In [24]:
sum(df_from_sql['bikes_docked'])

10648

Finally, let's clean up and close our database connection.

In [25]:
drop_db_query = "DROP DATABASE {db} ".format(db=db_name)

# Drop a database
cursor = con.cursor()
cursor.execute(drop_db_query)
cursor.close()


In [26]:
con.close()

